In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.device(0)

In [5]:
torch.cuda.get_device_name()

'Tesla T4'

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [7]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [8]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [9]:
train_dataset = dsets.MNIST(root='./data',
              train=True,
              transform = transforms.ToTensor(),
              download=True)

test_dataset = dsets.MNIST(root='./data',
              train=False,
              transform = transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.53MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.78MB/s]


In [10]:
import torch.utils.data as Data

In [11]:
train_loader = Data.DataLoader(dataset=train_dataset,
                batch_size=batch_size,
                shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,
                batch_size=batch_size,
                shuffle=False)

In [12]:
import torch.nn as nn

In [36]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2)
    )
    self.dense = nn.Sequential(
        nn.Linear(14*14*128, 1024),
        nn.ReLU(),
        nn.Linear(1024, 10)
    )

  def forward(self, x):
    out = self.conv(x)
    out = out.view(out.size(0), -1) # Flatten the output
    out = self.dense(out)
    return out

In [37]:
model = Model()
model.cuda()

Model(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [38]:
import torch.optim as optim
from torch.autograd import Variable

In [39]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images =  Variable(images.type(torch.cuda.FloatTensor))
    labels = Variable(labels.to(device)) # Correctly use original labels and move to device
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
     print("Epoch:%d, Batch:%d, Loss:%.4f"
     %(epoch+1, i+1, loss.data))

Epoch:1, Batch:100, Loss:0.1659
Epoch:1, Batch:200, Loss:0.0764
Epoch:1, Batch:300, Loss:0.0606
Epoch:1, Batch:400, Loss:0.0620
Epoch:1, Batch:500, Loss:0.0669
Epoch:1, Batch:600, Loss:0.1059
Epoch:2, Batch:100, Loss:0.0479
Epoch:2, Batch:200, Loss:0.0183
Epoch:2, Batch:300, Loss:0.0234
Epoch:2, Batch:400, Loss:0.0360
Epoch:2, Batch:500, Loss:0.0228
Epoch:2, Batch:600, Loss:0.0388
Epoch:3, Batch:100, Loss:0.0020
Epoch:3, Batch:200, Loss:0.0440
Epoch:3, Batch:300, Loss:0.0068
Epoch:3, Batch:400, Loss:0.0112
Epoch:3, Batch:500, Loss:0.0008
Epoch:3, Batch:600, Loss:0.0252
Epoch:4, Batch:100, Loss:0.0020
Epoch:4, Batch:200, Loss:0.0030
Epoch:4, Batch:300, Loss:0.0467
Epoch:4, Batch:400, Loss:0.0019
Epoch:4, Batch:500, Loss:0.0205
Epoch:4, Batch:600, Loss:0.0010
Epoch:5, Batch:100, Loss:0.0005
Epoch:5, Batch:200, Loss:0.0013
Epoch:5, Batch:300, Loss:0.0018
Epoch:5, Batch:400, Loss:0.0028
Epoch:5, Batch:500, Loss:0.0229
Epoch:5, Batch:600, Loss:0.0004


In [34]:
import torch

In [42]:
correct = 0
total = 0
for images, labels in test_loader:
  images = Variable(images.type(torch.cuda.FloatTensor))
  labels = labels.to(device)
  outputs = model(images)
  _, pred = torch.max(outputs.data, 1)
  correct += (pred == labels).sum()
  total += labels.size(0)

print("Accuracy:%.3f%%" %(100.0*float(correct)/float(total)))

Accuracy:98.870%
